In [94]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime

try:

    df_list = []

    for i in range(1, 31):
        try:
            filename = f'resources/transactions_summary ({i}).csv'
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f'Successfully loaded file {filename}')
            # Perform any data analysis or manipulation you need
        except FileNotFoundError:
            pass

    shake_data_df = pd.concat(df_list, ignore_index=True)
    shake_data_df['Date'] = pd.to_datetime(shake_data_df['Date']).dt.strftime('%Y-%m-%d')
    #Summary Spending
    summary_spending_df = shake_data_df[shake_data_df['Amount Debited'] > 1]
    summary_spending_df = summary_spending_df[['Amount Debited', 'Source / Destination','Date']]
    summary_spending_df=summary_spending_df.dropna()
    summary_spending_df = summary_spending_df[~summary_spending_df["Source / Destination"].isin(["PROGRESSIVE REHABILITA", "MASSAGE ADDICT","GOOGLE SERVICES","guevarch@gmail.com","FREEDOM MOBILE", "yansunique@yahoo.ca", "CITY CENTRE BAPTIST", "GOOGLE YouTubePremium"])]
    summary_spending_df = summary_spending_df[~summary_spending_df['Source / Destination'].str.contains(r'TOTAL HEALTH', regex=True)]
    df1 = pd.read_csv('resources/purchase_type_updated.csv')
    summary_spending_df = summary_spending_df.reset_index()
    summary_spending_df = summary_spending_df.merge(df1, on='Source / Destination', how='left')
    summary_spending_df.drop_duplicates(subset=['Date', 'Amount Debited','Source / Destination'], inplace=True)
    summary_spending_df = summary_spending_df.set_index('Amount Debited')
    # summary_spending_df.dropna(inplace=True)
    summary_spending_df.drop('index', axis=1, inplace=True)

    # Check for NaN values
    if summary_spending_df.isnull().values.any():
        print("There are NaN values in the DataFrame.")

        # Find rows with missing values in the "Type" column
        mask = summary_spending_df['Type'].isnull()
        missing_rows = summary_spending_df[mask]

        # Loop through the missing rows and fill them in with user input
        for index, row in missing_rows.iterrows():
            source_dest = row['Source / Destination']
            value2 = input(f"Enter a value to fill NaNs for '{source_dest}': ")
            
            # Replace the missing value with the user input
            summary_spending_df.loc[summary_spending_df['Source / Destination'] == source_dest, 'Type'] = summary_spending_df.loc[summary_spending_df['Source / Destination'] == source_dest, 'Type'].replace(np.nan, value2)
            
            # Print out the subset of the DataFrame where 'Source / Destination' is equal to 'source_dest'
            subset_df = summary_spending_df[summary_spending_df['Source / Destination'] == source_dest]
            print(subset_df)

    else:
        print("There are no NaN values in the DataFrame.")
    summary_spending_df = summary_spending_df.drop(['Unnamed: 0'], axis=1)
    # summary_spending_df = summary_spending_df.drop(['Unnamed: 0.1'], axis=1)
    summary_spending_df = summary_spending_df.reset_index()
    summary_spending_df = summary_spending_df.set_index('Type')
    # Save the DataFrame to a CSV file
    summary_spending_df.to_csv('resources/combined_summary.csv')
    # print(summary_spending_df.tail(10))

except FileNotFoundError:
    print("File not found. Please check the filename and path and try again.")


purchase_type_updated = pd.read_csv('resources/combined_summary.csv')
purchase_type_updated = purchase_type_updated.drop(['Date','Amount Debited'], axis=1)
purchase_type_updated = purchase_type_updated.drop_duplicates()
purchase_type_updated.to_csv('resources/purchase_type_updated.csv')
purchase_type_updated

Successfully loaded file resources/transactions_summary (1).csv
Successfully loaded file resources/transactions_summary (5).csv
There are no NaN values in the DataFrame.


,Type,Source / Destination
0,Food,METRO 854
1,Food,DERRY CONVENIENCE
2,Food,RCSS 1080
4,Food,DANFORTH FOOD MARKET
5,Misc,CLORE BEAUTY SUPPLY
...,...,...
1033,School,THOMPSON RIVERS UNIV WEB
1034,Misc,MTO RUS SO ECHANNEL
1038,Gas,ONROUTE 01177
1041,Misc,CARTERS 3402


In [114]:
import pandas as pd
# Read the CSV file and skip the first row (headers)
bmo_statement = pd.read_csv('resources/statement (1).csv', skiprows=1)
exclude_strings = ['TRSF FROM/DE ACCT/CPT 3978-XXXX-729', 'RBC LIFE INSURANCE CO. MISSISSAUGA ON','AVIVA GENERAL INSURANC MARKHAM ON', 'ANNUAL CARD FEE', 'ANNUAL CARD FEE REBATE']
# Exclude rows containing any of the exclude_strings in the 'Description' column
bmo_statement = bmo_statement[~bmo_statement['Description'].str.contains('|'.join(exclude_strings))]
columns_to_remove = ['Item #', 'Card #', 'Posting Date']
bmo_statement = bmo_statement.drop(columns=columns_to_remove)
# Print the DataFrame without headers
bmo_statement



,Transaction Date,Transaction Amount,Description
0,20230928,160.00,COLLEGE OF EARLY CHILD TORONTO ON
1,20231001,126.00,COSTCO WHOLESALE W531 MISSISSAUGA ON
6,20231005,44.46,COSTCO WHOLESALE W531 MISSISSAUGA ON
7,20231005,4.62,COSTCO WHOLESALE W531 MISSISSAUGA ON
8,20231007,203.41,COSTCO WHOLESALE W531 MISSISSAUGA ON
9,20231007,1.58,COSTCO WHOLESALE W531 MISSISSAUGA ON
11,20231013,350.53,COSTCO WHOLESALE W531 MISSISSAUGA ON
12,20231015,180.00,DONORDRIVE4DOROTHY MISSISSAUGA ON
16,20231017,952.43,LENOVO CANADA MARKHAM ON
18,20231021,254.13,COSTCO WHOLESALE W531 MISSISSAUGA ON


In [115]:
combined_summary = pd.read_csv('resources/combined_summary.csv')
combined_summary

,Type,Amount Debited,Source / Destination,Date
0,Food,18.78,METRO 854,2022-02-23
1,Food,4.05,DERRY CONVENIENCE,2022-02-23
2,Food,101.42,RCSS 1080,2022-02-24
3,Food,34.92,METRO 854,2022-02-26
4,Food,44.36,DANFORTH FOOD MARKET,2022-03-12
...,...,...,...,...
1050,Food,10.50,OSMOWS,2023-10-12
1051,Misc,113.43,BEST BUY 622,2023-10-14
1052,Gas,44.67,CDN TIRE GASBAR 01311,2023-10-14
1053,Food,29.88,METRO 854,2023-10-16
